In [1]:
import os
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from resnet import ResNet18
from network_func import *
from optimizer import *
from training import *
from useful_functions import *
from CIFAR10_data_preparation import *

In [2]:
n=5
I=np.eye(n)
one=np.ones(n)
R=np.outer(one,one)/n
A,C=di_ring(n=n)
B=get_B(A=A,u=2*n,n=n)
show_row(A)
_,__=test_row(A)
print(f"{_:.4e}, {__:.4e}")
h_data,y_data,X_test,y_test=cifar10_prepare_node_5_hard_shuffled()

A的第二大特征值: 0.7360318786212677
A的beta: 0.810217360924142
A的spectral gap: 0.18978263907585802
A的kappa: 2.000000000000005
S_A是: 39.898193112709905 

1.8413e+00, 2.7000e+01


In [3]:
def compute_accuracy_with_weighted_model_resnet18(model_list, X_test, y_test, weights, criterion, testloader):
    # 确保 X_test 和 y_test 在相同的设备上
    device = next(model_list[0].parameters()).device

    # Step 1: 计算加权模型的参数
    avg_model = ResNet18(num_classes=10).to(device)
    avg_state_dict = avg_model.state_dict()

    # 初始化加权的参数字典
    weighted_state_dict = {key: torch.zeros_like(param).to(device) for key, param in avg_state_dict.items()}

    # 汇总加权参数
    for model, weight in zip(model_list, weights):
        state_dict = model.state_dict()
        for key in weighted_state_dict.keys():
            weighted_state_dict[key] += state_dict[key].to(device) * weight

    # 计算总权重
    total_weight = sum(weights)
    weighted_avg_state_dict = {key: value / total_weight for key, value in weighted_state_dict.items()}

    # 将加权参数加载到新模型中
    avg_model.load_state_dict(weighted_avg_state_dict)

    # 确保测试数据在正确的设备上
    X_test = X_test.to(device)
    y_test = y_test.to(device)

    # Step 2: 评估加权模型的准确性
    avg_model.eval()
    sum_loss = 0.0
    correct = 0.0
    total = 0
    
    with torch.no_grad():
        outputs = avg_model(X_test)
        loss = criterion(outputs, y_test)
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += y_test.size(0)
        correct += predicted.eq(y_test.data).cpu().sum()

    val_loss = sum_loss / len(testloader)
    val_accu = 100. * correct / total
    
    return val_loss, val_accu


In [4]:
import importlib

import training
importlib.reload(training)
from training import *

In [5]:
Sum_l1, Sum_a1 = six_GPU_train_PullSum(
    n=n,
    A=A,
    B=B,
    model_class=ResNet18,
    seed_for_model=49,
    criterion_class=nn.CrossEntropyLoss,
    epochs=50,
    lr=3e-3,
    batch_size=128,
    X_train_data=h_data,
    y_train_data=y_data,
    X_test_data=X_test,
    y_test_data=y_test,
    compute_accuracy=compute_accuracy_with_weighted_model_resnet18,
    show_graph=True
)

/root/miniconda3/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


optimizer初始化成功!
optimizer初始化成功!


Training Progress:   0%|          | 0/50 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cuda:0! (when checking argument for argument mat in method wrapper_CUDA_addmv_)